In [ ]:
#라이브러리 불러오기
import os
import zipfile
import pandas as pd
import shutil
import os
import glob
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pandas as pd

from pathlib import Path
from collections import OrderedDict


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset,DataLoader

import torch.nn as nn
import torch.optim as optim


In [ ]:
#GPU 확인하기
train_on_gpu = torch.cuda.is_available()


if not train_on_gpu:
    device = 'cpu'
    print('CUDA is not available.  Training on CPU ...')
else:
    device = 'cuda'
    print('CUDA is available!  Training on GPU ...')
    

In [ ]:
# from sklearn.model_selection import train_test_split

# cvs 파일 읽어서 DataFrame으로 저장
df_train = pd.read_csv('/kaggle/input/chanbabo/train.csv')
df_val = pd.read_csv('/kaggle/input/chanbabo/val.csv')

df_train


In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class Classification_Dataset(Dataset):
    def __init__(self, csv, mode, transform=None):
        self.csv = csv.reset_index(drop=True)  # random으로 섞인 데이터의 인덱스를 reset 시켜서 다시 부여한다.
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return self.csv.shape[0]  # csv 파일의 행 개수 == 데이터 개수

    def __getitem__(self, index):   
        row = self.csv.iloc[index]                       # 주어진 index에 대한 데이터 뽑아오기
        st = row.names.replace('\\', '/')
        if self.mode == 'train':
          image = Image.open(f'/kaggle/input/chanbabo/train/{st}').convert('RGB') # 파일경로로 부터 이미지를 읽고 rgb로 변환하기
        elif self.mode == 'valid':
          image = Image.open(f'/kaggle/input/chanbabo/val/{st}').convert('RGB') # 파일경로로 부터 이미지를 읽고 rgb로 변환하기
        target = torch.tensor(self.csv.iloc[index].labels).long()
        
        if self.transform:
            image = self.transform(image) # 이미지에 transform 적용하기

        return image, target  # 이미지와 target return하기

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

train_transforms = transforms.Compose([transforms.Resize((224,224)), #사이즈키운거
                                       transforms.RandomRotation(30), #돌리기
#                                        transforms.RandomCrop(224),
#                                        transforms.ColorJitter(contrast=0.2, saturation=0.2, hue=0.2),#색상 변형
# #                                        transforms.RandomResizedCrop(224),        
                                       transforms.RandomHorizontalFlip(), #좌우 반전
                                       transforms.RandomVerticalFlip(), #좌우 반전
# #                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225]),
#                                        transforms.Normalize((0.5,), (0.5,)),
                                       transforms.RandomErasing() #구멍내기
                                       ])
    
test_transforms = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
#     transforms.Normalize((0.5,), (0.5, ))
    ])


In [ ]:
train_data = Classification_Dataset(df_train, 'train', transform=train_transforms)
valid_data = Classification_Dataset(df_val, 'valid', transform=test_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)

In [ ]:
def imshow(img):
    # img = img / 2 + 0.5  # unnormalize
    MEAN = np.array([0.485 , 0.456, 0.406])
    STD = np.array([0.229, 0.224 , 0.225])
    img = img * STD[:, None, None] + MEAN[:, None, None]   # unnormalize
    plt.imshow(np.clip(np.transpose(img, (1, 2, 0)), 0, 1))

In [ ]:
data, label =  next(iter(train_loader))
print(data.shape, label.shape)

fig = plt.figure(figsize=(15, 4))
# display 20 images
data = data.numpy()
for idx in np.arange(20):
    ax = fig.add_subplot(2, int(20/2), idx+1, xticks=[], yticks=[])
    imshow(data[idx])


In [ ]:
# !pip install timm

In [ ]:
#전이 학습을 위해 모델들 확인
import timm
# timm.list_models(pretrained=True)

In [ ]:
#전이학습 모델 불러오기
model = timm.create_model('efficientnet_b3.ra2_in1k', pretrained=True, num_classes=5)
model.to(device)

In [ ]:

import torch.nn.functional as F
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
# ,weight_decay=0.09

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')


In [ ]:

#에폭이랑 로스 저장할 변수 선어.
n_epochs = 100
valid_loss_min = np.inf
valid_acc_min = 0 
train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)
train_acc = torch.zeros(n_epochs)
valid_acc = torch.zeros(n_epochs)

for e in range(0, n_epochs):

   
    #######
    # 학습 #
    #######
    model.train()
    
    for data, labels in tqdm(train_loader):
        
        #데이터들 gpu로 보내주기. 
        data, labels = data.to(device), labels.to(device)
        
        #옵티마지어 초기화
        optimizer.zero_grad()
        #데이터를 모델에 보내기.
        logits = model(data)
       
        #로스 계산
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        #학습 로스 계산, 정답 확인
        train_loss[e] += loss.item()
        
        ps = F.softmax(logits, dim=1)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.reshape(top_class.shape)
        train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
    
    train_loss[e] /= len(train_loader)
    train_acc[e] /= len(train_loader)
        
        
    ######    
    #검증 #
    ######
    with torch.no_grad(): 
        model.eval()
        for data, labels in valid_loader:
                
            #데이터들 gpu로 보내주기. 
            data, labels = data.to(device), labels.to(device)
            #데이터를 모델에 보내기.
            logits = model(data)
            
            #로스 계산
            loss = criterion(logits, labels)
            
            #검증 로스 계산, 정답 확인
            valid_loss[e] += loss.item()
                  
            ps = F.softmax(logits, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.reshape(top_class.shape)
            valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
    
    #검증 로스 계산
    valid_loss[e] /= len(valid_loader)
    valid_acc[e] /= len(valid_loader)
    
    #스케쥴러를 사용하기
    scheduler.step(valid_loss[e])
        
    #학습/검증 출력. 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss[e], valid_loss[e]))
    print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
        e, train_acc[e], valid_acc[e]))
    

    #모델 저장하기
    if valid_loss[e] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
       valid_loss_min,
       valid_loss[e]))
        torch.save(model.state_dict(), './model1.pt')
        valid_loss_min = valid_loss[e]

In [ ]:
#로스 그래프 확인하기.
plt.ylim([0, 2.5])     # Y축의 범위: [ymin, ymax]

plt.plot(train_loss)
plt.plot(valid_loss)

In [ ]:
plt.plot(train_acc)
plt.plot(valid_acc)

In [ ]:
class test_dataset(Dataset):
    def __init__(self,imgpath, transform=None):
        
        self.imgpath = imgpath
        self.transform = transform
        
    def __len__(self):
        return len(self.imgpath)
    
    def __getitem__(self,idx):
        x = self.transform(Image.open(self.imgpath[idx]).convert('RGB'))
        
        return x

In [ ]:
pip install natsort

In [ ]:
import natsort

test_set = sorted(glob.glob('/kaggle/input/test-ex2/test/*'))  # 이미지 파일의 확장자에 맞게 수정해주세요
# #디렉토리 파일명 가져오기
test_set = natsort.natsorted(test_set)

#학습한 모델 불러오기.
model = timm.create_model('efficientnet_b2.ra_in1k', pretrained=False, num_classes=5)
model.to(device)
model.load_state_dict(torch.load('/kaggle/input/models/model1(eff_b3).pt'))

test_data = test_dataset(test_set, test_transforms)
test_DataLoader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
#테스트 결과 뽑아보기.
pred = torch.zeros(len(test_images))
cnt =0

model.eval()
with torch.no_grad(): 
    for i,images in enumerate(test_DataLoader):
        images = images.to(device)
        logits = model(images)
        
        ps = F.softmax(logits, dim=1)
        top_p, top_class = ps.topk(1, dim=1)
        #print(top_class)
        
        for idx in np.arange(len(top_class)):
           # print(cnt)
            pred[cnt] = top_class[idx]
            cnt+=1

In [ ]:
pred = pred.reshape(-1).to(dtype=torch.long, device=device)

In [ ]:
pred

In [ ]:
nn = []
for i in range(15):
    nn.append(f'test\{i}.jpg')

In [ ]:
id=[]
category = np.zeros(15)
for i in range(len(test_set)):
    category[i] = (pred[i])
    
df = pd.DataFrame()
df['names'] = nn
df['labels'] = category
df.to_csv('submission_1111.csv',index=False)

In [ ]:
df